# End to End Deep Learning Project Using Simple RNN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
import gc
gc.collect()

In [ ]:
## load the imdb dataset
max_features = 10000  # vocabulary(number of words to consider as features)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

print(f'Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}')
print(f'Testing data shape: {X_test.shape}, Testing labels shape: {y_test.shape}')

In [ ]:
## Inspect a sample review
sample_review = X_train[0]
sample_label = y_train[0]

print(f'Sample review (as word indices): {sample_review},\n{sample_label}')

In [ ]:
# Mapping of word indices to words
word_index = imdb.get_word_index()
print(f'word_index: {list(word_index.items())[:10]}')  # Print first 10 entries

reverse_word_index = {value: key for (key, value) in word_index.items()}
print(f'reverse_word_index: {list(reverse_word_index.items())[:10]}')  # Print first 10 entries

In [ ]:
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in sample_review])
print(f'Decoded review: {decoded_review}')

In [ ]:
# Pad sequences to ensure uniform input length
maxlen = 500  # maximum length of a review

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)


In [ ]:
print(f'Padded training data shape: {X_train.shape}')

In [ ]:
## Train a simple RNN model

model = Sequential()

model.add(Embedding(max_features, 128))  # Embedding layer
model.add(SimpleRNN(128, activation='relu'))  # Simple RNN layer
model.add(Dense(1, activation='sigmoid'))  # Output layer

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
## Create an instance of EarlyStopping Callback
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
## Train a simple RNN model
model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

In [ ]:
## Save the model
model.save('simple_rnn_imdb.h5')